# 基于医嘱明细医学编码数据的深度学习患者相似度度量和查找方法及案例报告

                                           李师众   2018.12.19

# 1）数据说明

In [2]:
算法使用了实施部提取出的《visit_order.csv》住院患者医嘱明细表数据。取前五条数据作为示例展示，数据结构如下：

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,HOSPITAL_CODE,PATIENT_ID,PATIENT_NO,INPATIENT_NO,SERIAL_NO,ITEM_SNOMED,SN_COMMENT,ITEM_CODE,MO_NAME,...,UPDATED_ON,SEQ_ID,SYS_EMP_NO,DATA_VERSION,SYS_MO_NAME,SYS_DOSE_UNIT,SYS_UNIT,SYS_USE_NAME,ITEM_CLASS,normalized_item_code
0,0,12440100455344205E,0081041264,581043,ZY070000581043,196256095,NaN,NaN,Y00000004991,善存片,...,05-6月 -18 03.15.09.000000 下午,43813,NaN,0,NaN,NaN,NaN,口服,西药,XA11AA-XD185-01
1,1,12440100455344205E,0081265768,595649,ZY010000595649,194027186,NaN,NaN,F00002227295,一次性使用输液器（超低密度聚乙烯精密输液器（带螺口））,...,05-6月 -18 03.15.09.000000 下午,44335,NaN,0,NaN,NaN,NaN,NaN,治疗,NaN
2,2,12440100455344205E,0080234445,353163,ZY090000353163,185582066,NaN,NaN,18435,九项呼吸道感染病原体IgM抗体检测,...,05-6月 -18 03.15.09.000000 下午,44807,NaN,0,NaN,NaN,NaN,NaN,检验,18435
3,3,12440100455344205E,0000764987,606770,ZY010000606770,202544697,NaN,NaN,F00002204394,动脉采血,...,05-6月 -18 03.15.09.000000 下午,44824,NaN,0,NaN,NaN,NaN,NaN,治疗,NaN
4,4,12440100455344205E,0000749211,321757,ZY060000321757,85493264,NaN,NaN,F00002204391,静脉采血,...,05-6月 -18 03.15.09.000000 下午,45318,NaN,0,NaN,NaN,NaN,NaN,治疗,NaN


In [3]:
由于算法主要需要用到患者PATIENT_ID、INPATIENT_NO、ITEM_CODE属性构建关联，故从中提取算法主要用到的属性以部分冗余属性作为参考。
取前五条数据作为示例展示，数据结构如下：

,PATIENT_ID,INPATIENT_NO,ITEM_CODE,MO_NAME,DATE_BGN,DATE_END,OPEN_TIME,ITEM_CLASS,normalized_item_code
0,0081041264,ZY070000581043,Y00000004991,善存片,2018/1/5 星期五 0:01:00,2018/1/12 星期五 10:42:19,2018/1/5 星期五 16:46:54,西药,XA11AA-XD185-01
1,0081265768,ZY010000595649,F00002227295,一次性使用输液器（超低密度聚乙烯精密输液器（带螺口））,2017/12/8 星期五 9:54:39,2017/12/8 星期五 10:25:05,2017/12/8 星期五 9:54:41,治疗,NaN
2,0080234445,ZY090000353163,18435,九项呼吸道感染病原体IgM抗体检测,2017/8/16 星期三 15:17:42,0001/1/1 星期一,2017/8/16 星期三 15:17:42,检验,18435
3,0000764987,ZY010000606770,F00002204394,动脉采血,2018/3/29 星期四 17:41:36,0001/1/1 星期一,2018/3/29 星期四 17:41:36,治疗,NaN
4,0000749211,ZY060000321757,F00002204391,静脉采血,2013/2/16 星期六 16:32:29,0001/1/1 星期一,2013/2/16 星期六 16:32:29,治疗,NaN


# 2）数据处理流程

对数据根据患者唯一ID及其住院号进行分组聚合操作。分组后可依据患者ID及住院号取出相关数据，其中一名患者的一次住院数据示例如下：

In [14]:
# 数据示例
df_doctorOrderByID.get_group(('0000014122', 'ZY070000138161'))

,PATIENT_ID,INPATIENT_NO,ITEM_CODE,MO_NAME,DATE_BGN,DATE_END,OPEN_TIME,ITEM_CLASS,normalized_item_code
3523,0000014122,ZY070000138161,Y00000018577,奥德金注射液,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:45:14,西药,XV06DX-XX140-02
7466,0000014122,ZY070000138161,Y00000019561,(益康宁)前列地尔注射液#,2015/8/5 星期三 0:01:00,2015/8/14 星期五 9:18:35,2015/8/5 星期三 11:05:25,西药,XC01EA-XQ015-02
14714,0000014122,ZY070000138161,17100,CX3生化八项,2015/8/4 星期二 10:42:19,0001/1/1 星期一,2015/8/4 星期二 10:42:19,检验,17100
15065,0000014122,ZY070000138161,999,明天补液后出院,2015/8/13 星期四 9:49:42,0001/1/1 星期一,2015/8/13 星期四 9:49:42,预约出院,999
16860,0000014122,ZY070000138161,F00002227295,一次性使用输液器（超低密度聚乙烯精密输液器（带螺口））,2015/8/4 星期二 11:22:36,2015/8/14 星期五 9:18:35,2015/8/4 星期二 11:22:37,治疗,NaN
21388,0000014122,ZY070000138161,17943,EB病毒抗体测定,2015/8/4 星期二 10:43:00,0001/1/1 星期一,2015/8/4 星期二 10:43:00,检验,17943
23959,0000014122,ZY070000138161,F00002204391,静脉采血,2015/8/4 星期二 10:42:20,0001/1/1 星期一,2015/8/4 星期二 10:42:20,治疗,NaN
25392,0000014122,ZY070000138161,Y00000018274,0.9%氯化钠注射液&#@,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:45:14,西药,XB05XA-XL211-02
31718,0000014122,ZY070000138161,Y00000018477,盖三淳胶丸#,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:46:18,西药,XA11CC-XG111-01
32471,0000014122,ZY070000138161,16179,酶八项,2015/8/4 星期二 10:42:19,0001/1/1 星期一,2015/8/4 星期二 10:42:19,检验,16179


以下为从数据中获取到的item_code集合数量、患者最多访问次数、患者总数量等参数：

In [48]:
#  全部非重复的item_code数组
list_itemCode = df_doctorOrder_exact['ITEM_CODE'].unique().tolist()
count_uniqueItemCode = len(list_itemCode)
print(count_uniqueItemCode)

2536


In [30]:
#  病人最大访问次数
count_maxVisit = df_doctorOrder_exact.drop_duplicates('INPATIENT_NO').groupby('PATIENT_ID').size().max()
print(count_maxVisit)

15


In [35]:
#  病人数量
count_patientNum = df_doctorOrder_exact['PATIENT_ID'].unique().size
print(count_patientNum)

893


In [52]:
经过一系列筛选及填充步骤，将数据处理成（患者数，最大访问数，单次访问全部医学编码one-hot聚合向量）的tensor形式，数据结构如下：

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


# 3）针对患者单次访问的医学编码稀疏向量进行嵌入

In [84]:
构建AutoEncoder模型，对患者医学编码稀疏向量进行embedding训练来获取多次访问医学编码的密集向量表达。
训练输出如下，可以看到训练后验证集精度可达到92%：

Train on 10716 samples, validate on 2679 samples
Epoch 1/100
10716/10716 [==============================] - 9s 831us/step - loss: 0.0035 - acc: 0.0029 - val_loss: 0.0020 - val_acc: 0.0000e+00
Epoch 2/100
  192/10716 [..............................] - ETA: 8s - loss: 0.0028 - acc: 0.0000e+00

/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


10716/10716 [==============================] - 9s 855us/step - loss: 0.0024 - acc: 0.0220 - val_loss: 0.0018 - val_acc: 3.7327e-04
Epoch 3/100
10716/10716 [==============================] - 10s 954us/step - loss: 0.0021 - acc: 0.2125 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 4/100
10716/10716 [==============================] - 9s 805us/step - loss: 0.0019 - acc: 0.4560 - val_loss: 0.0016 - val_acc: 0.9227
Epoch 5/100
10716/10716 [==============================] - 10s 908us/step - loss: 0.0019 - acc: 0.6337 - val_loss: 0.0015 - val_acc: 0.9224
Epoch 6/100
10716/10716 [==============================] - 10s 959us/step - loss: 0.0018 - acc: 0.6869 - val_loss: 0.0015 - val_acc: 0.9220
Epoch 7/100
10716/10716 [==============================] - 10s 896us/step - loss: 0.0018 - acc: 0.7429 - val_loss: 0.0015 - val_acc: 0.9235
Epoch 8/100
10716/10716 [==============================] - 9s 875us/step - loss: 0.0017 - acc: 0.7903 - val_loss: 0.0015 - val_acc: 0.9227
Epoch 9/100
10716/10716 [==

获取到训练后的嵌入层，以下为一个患者单次访问的密集向量表达示例：

In [88]:
embedding_layer_model = keras.Model(inputs=model.input,
                                     outputs=model.get_layer('dense_38').output)
#以这个model的预测值作为输出
embedding_output = embedding_layer_model.predict(data_2D_itemCode)
 
print(embedding_output.shape)
print(embedding_output[0])

(13395, 128)
[0.         0.22466484 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 2.0885463  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.45330447 0.         0.         0.         0.
 0.         0.13245502 0.         0.         0.         0.
 0.         0.         0.40092218 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.17747174 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.03939337
 0.         0.         0.         0.         0.         0.
 0.         1.2983484  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0

# 4）根据生成的患者多次访问医学编码密集向量表达进行相似度计算

In [130]:
使用893个患者的多次访问医学编码密集向量进行患者两两间的相似度计算，生成（893，893）形式的患者相似度矩阵，输出示例如下：

[[0.9999999  0.80015904 0.5699328  ... 0.96690786 0.9623244  0.98765254]
 [0.80015904 0.99999976 0.5499156  ... 0.7994744  0.7874751  0.8070792 ]
 [0.5699328  0.5499156  1.         ... 0.5624945  0.5586596  0.57381785]
 ...
 [0.96690786 0.7994744  0.5624945  ... 0.99999976 0.9921602  0.9828981 ]
 [0.9623244  0.7874751  0.5586596  ... 0.9921602  0.9999999  0.97769034]
 [0.98765254 0.8070792  0.57381785 ... 0.9828981  0.97769034 0.9999999 ]]


然后根据患者列表，按顺序在相似矩阵中查找每位患者的相似度最高的患者在患者列表中的序号，生成与患者列表顺序一致的对应最相似患者列表。
下面输入示例为取患者列表第一个患者的patient_ID,及从最相似患者列表中取出的其对应最相似患者的patient_ID，
即完成了在目前数据库中，一个指定患者的相似患者查找流程，以下为两个患者的ID：

In [135]:
selected_current_patientID = list_patientID[0]
selected_current_mostSimilar_patientID = list_patientID[list_mostSimilairPatient[0]]
print(selected_current_patientID)
print(selected_current_mostSimilar_patientID)

0000007630
0081344080


In [146]:
我们指定的患者列表中的首个患者数据如下：

(188, 9)
        PATIENT_ID    INPATIENT_NO     ITEM_CODE                      MO_NAME  \
1248    0000007630  ZY040000336335  Y00000020121                0.9%氯化钠注射液&#@   
1409    0000007630  ZY040000336335           999                 测BP、P、R、SPO2   
3370    0000007630  ZY040000336335  Y00000019982                      明可欣针**#   
5829    0000007630  ZY040000336335  F00002204397            静脉输液(连续输液第二组起每组收)   
9088    0000007630  ZY040000336335  F00002218596               一次性注射器30ML/因特圣   
9543    0000007630  ZY040000336335  Y00000001533                     消脱止-M片**   
10008   0000007630  ZY040000336335  F00002218595               一次性注射器20ML/因特圣   
10373   0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
11185   0000007630  ZY040000336335  Y00000000673                 葡萄糖氯化钠注射液&#@   
12699   0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
15535   0000007630  ZY040000336335  F00002204346                         院内会诊   
16464   0000007630 

In [147]:
查找到的数据库中该指定患者最相似患者数据如下：

(158, 9)
        PATIENT_ID    INPATIENT_NO     ITEM_CODE                      MO_NAME  \
3976    0081344080  ZY010000608852           999                           普食   
5647    0081344080  ZY010000608852  F00002213154                        引流管引流   
7340    0081344080  ZY010000608852  Y00000018919                  地佐辛注射液(加罗宁)   
7815    0081344080  ZY010000608852  F00002218595               一次性注射器20ML/因特圣   
8503    0081344080  ZY010000608852  Y00000019941                       阿托品注射液   
10032   0081344080  ZY010000608852  F00002212422                        黄头真空管   
12727   0081344080  ZY010000608852  F00002204400                       住院静脉输液   
15765   0081344080  ZY010000608852  Y00000020216            0.9%氯化钠注射液(科伦)&#@   
20139   0081344080  ZY010000608852  F00002212422                        黄头真空管   
22066   0081344080  ZY010000608852  F00002204352                         Ⅰ级护理   
24396   0081344080  ZY010000608852           999                        午夜后禁食   
35604   0081344080 

将两名患者共有的相同医学编码进行聚合输出，示例如下：

In [158]:
df_compareSimilar_dropDuplication = df_compareSimilar.drop_duplicates(['ITEM_CODE','MO_NAME'])
print(df_compareSimilar_dropDuplication.shape)
print(df_compareSimilar_dropDuplication)

(170, 8)
    PATIENT_ID_x  INPATIENT_NO_x     ITEM_CODE                      MO_NAME  \
0     0000007630  ZY040000336335  Y00000020121                0.9%氯化钠注射液&#@   
2     0000007630  ZY040000336335           999                 测BP、P、R、SPO2   
3     0000007630  ZY040000336335  Y00000019982                      明可欣针**#   
7     0000007630  ZY040000336335  F00002204397            静脉输液(连续输液第二组起每组收)   
37    0000007630  ZY040000336335  F00002218596               一次性注射器30ML/因特圣   
41    0000007630  ZY040000336335  Y00000001533                     消脱止-M片**   
42    0000007630  ZY040000336335  F00002218595               一次性注射器20ML/因特圣   
96    0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
111   0000007630  ZY040000336335  Y00000000673                 葡萄糖氯化钠注射液&#@   
114   0000007630  ZY040000336335  F00002204346                         院内会诊   
117   0000007630  ZY040000336335  Y00000002188                     可必特溶液**#   
118   0000007630  ZY040000336335  F00000181